In [ ]:
import time
import urllib2
import pandas as pd
import requests
from bs4 import BeautifulSoup
from multiprocessing.dummy import Pool as ThreadPool

tv = pd.read_csv('C:/Users/Administrator/Desktop/tv.csv',header = None,encoding = 'gbk')
tv_list = [i.encode('utf-8') for i in list(pd.unique(tv.iloc[:,0]))]

dianying = pd.read_csv('C:/Users/Administrator/Desktop/dianying.csv',header = None,encoding = 'gbk')
dianying_list = [i.encode('utf-8') for i in list(pd.unique(dianying.iloc[:,0]))]

v = pd.read_csv('C:/Users/Administrator/Desktop/v.csv',header = None,encoding = 'gbk')
v_list = [i.encode('utf-8') for i in list(pd.unique(v.iloc[:,0]))]

dongman = pd.read_csv('C:/Users/Administrator/Desktop/dongman.csv',header = None,encoding = 'gbk')
dongman_list = [i.encode('utf-8') for i in list(pd.unique(dongman.iloc[:,0]))]

kankan = pd.read_csv('C:/Users/Administrator/Desktop/kankan.csv',header = None,encoding = 'gbk')
kankan_list = [i.encode('utf-8') for i in list(pd.unique(kankan.iloc[:,0]))]

start = time.time()
headers = { 
            'User-Agent':'User-Agent:Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/52.0.2743.116 Safari/537.36',
            'Cookie':'BAIDUID=D2154FDB12879492D61C57735FCC52FD:FG=1'
            }


def rank_nos(key_word,page):
    url = url = 'https://www.baidu.com/s?wd='+key_word+'&pn='+str(page)+"&oq="+key_word+"&ie=utf-8&usm=3&rsv_pq=e4618f0f0000ff3a"
    
    response = response = requests.get(url,headers = headers).text

    soup = BeautifulSoup(response,'html.parser')
    patrent = soup.find_all('div',tpl = 'se_com_default')
    list_i = []
    for i in patrent:
        if "2345.com" in str(i.find_all('a',class_ = 'c-showurl')[0].contents):
            list_i.append(int(i['id']))
    return min(list_i) if list_i else 99

def rank_withs(key_word,page):
    url = url = 'https://www.baidu.com/s?wd='+key_word+'&pn='+str(page)+"&oq="+key_word+"&ie=utf-8&usm=3&rsv_pq=e4618f0f0000ff3a"
    
    response = response = requests.get(url,headers = headers).text 

    soup = BeautifulSoup(response,'html.parser')
    patrent = soup.find_all('div',tpl = 'se_com_default')
    list_i = []
    for i in patrent:
        if "<b>2345</b>" in str(i.find_all('a',class_ = 'c-showurl')[0].contents):
            list_i.append(int(i['id']))
    return min(list_i) if list_i else 99

def rank(key_word):
    rank_id = 0
    if '2345' in key_word:
        rank_id=rank_withs(key_word,page=0)
    else:
        if rank_nos(key_word,page=0)<99:
            rank_id=rank_nos(key_word,page=0)
        elif rank_nos(key_word,page=0)==99 and rank_nos(key_word,page=10)<99:
            rank_id=rank_nos(key_word,page=10)
        elif rank_nos(key_word,page=10)==99:
            rank_id=rank_nos(key_word,page=20)
    return rank_id

In [ ]:
pool = ThreadPool(8)
result_tv = pool.map(rank,tv_list)
result_diangying = pool.map(rank,dianying_list)
result_v = pool.map(rank,v_list)
result_dongman = pool.map(rank,dongman_list)
result_kankan = pool.map(rank,kankan_list)
pool.close()
pool.join()
print("Time: {:.2f} seconds".format(time.time() - start))

df_tv = pd.DataFrame()
df_tv['search_word'] = tv_list
df_tv['rank'] = result_tv
df_tv.to_csv(u'C:/Users/Administrator/Desktop/tv排名.csv',encoding = 'gbk')

df_dy = pd.DataFrame()
df_dy['search_word'] = dianying_list
df_dy['rank'] = result_dianying
df_dy.to_csv(u'C:/Users/Administrator/Desktop/电影排名.csv',encoding = 'gbk')

df_v = pd.DataFrame()
df_v['search_word'] = v_list
df_v['rank'] = result_v
df_v.to_csv(u'C:/Users/Administrator/Desktop/v排名.csv',encoding = 'gbk')

df_d = pd.DataFrame()
df_d['search_word'] = dongman_list
df_d['rank'] = result_dongman
df_d.to_csv(u'C:/Users/Administrator/Desktop/动漫排名.csv',encoding = 'gbk')

df_k = pd.DataFrame()
df_k['search_word'] = kankan_list
df_k['rank'] = result_kankan
df_k.to_csv(u'C:/Users/Administrator/Desktop/kankan排名.csv',encoding = 'gbk')

In [ ]:
rank('抗日战争')